"Descent into Collective Entanglement"

This thesis work is to explore the possibility of faster calculations for calculating "Collectibility" for a given density matrix for a given multipartite system. In this work, we mainly focus on the 3*3 system.

We we wilm be trying out gradient descent algorithms for the purpose of finding the optimum "separable state set" to get a minimum value of collectibility. This minima, in return, will indicate if the system is entangled.

We will start with testing on known classes of density matrices (eg, werner states) which have known bounds as a sanity check of the approach.

Steps - 
1. Function working with unitary matrix. It will have a function to calculate random unitary matrix using Zyczkowski's method. ✅
2. Moreover, it needs to calculate the partial differential of U wrt an individual parameter. And then return the gradient for the whole parameters vector. ✅
3. Decide on the step size.✅
4. Work on generating density matrices - Random Density matrices and werner states.✅
5. Work out the collectibility function for given parameters.✅
6. After calling the gradient calculating function and deciding on the step size, we calculate the value of Collectibility for the next step.✅
7. We find the local minima, and do this procedure more times, to search fort he global minima.
8. Visualize the results for plotting graphs and getting a visual sense of the work.
9. Compare with brute force methods by searching for the optimum solution (minima) by random search.

In [424]:
# Library imports

import random
import cmath,math
from pprint import pprint
import copy
import time
import itertools

#numpy and matplot
import matplotlib.pyplot as plt
import numpy as np

#Qutip imports
from qutip import (Qobj, about, basis, coherent, coherent_dm, create, 
                  destroy,expect, fock, fock_dm, mesolve, qeye, sigmax, 
                  sigmay,sigmaz, tensor, thermal_dm,dimensions,
                  rand_dm,rand_unitary,partial_transpose,
                  hadamard_transform,projection,phase_basis)

import qutip

from functools import reduce

In [527]:
# List of intial states - 

#Random rho generator - using banacki's method (and maybe using Qobj)
def Mutually_Orthogonal_state_generator(system):
    '''
    Initially, the assumption is that all Hilbert spaces of the subsystem are of 
    the same dimension.
    Should I pad the lower dimension subsystems with zero columns??
    '''
    N = min(system)
    MO_Basis_set = []
    for subsys_dim in system:
        MO_Basis_subset = [basis(subsys_dim, n) for n in range(subsys_dim)]
        MO_Basis_set.append(MO_Basis_subset)
    return MO_Basis_set

def Seperable_state_set_generator(MO_Basis_set):
    #Initially, the assumption is that all Hilbert spaces of the subsystem are of 
    # the same dimension.
    
    Seperable_basis_set = []
    N = max([len(subsys) for subsys in MO_Basis_set])
    for n in range(N): 
        Seperable_basis_set.append(tensor([subsys[n] for subsys in MO_Basis_set]))
    return Seperable_basis_set

system = [3,3]        
Sep_state_set = Seperable_state_set_generator(Mutually_Orthogonal_state_generator(system))
print(Sep_state_set)

[Quantum object: dims = [[3, 3], [1, 1]], shape = (9, 1), type = ket
Qobj data =
[[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], Quantum object: dims = [[3, 3], [1, 1]], shape = (9, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]], Quantum object: dims = [[3, 3], [1, 1]], shape = (9, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]]]


In [426]:
# Generating rho (random and werner)
# Generating Random Density Matrices - few options - 

# 1. Random Density matrix using built in Qutip functions - 
rand_dm(size)

# 2. Generating Werner State

def Probability_generator(n):
    '''
    Generates list of Probabilites with n elements, for by partitioning 1 with each step. 
    Hence, adding up to 1.
    '''
    Probs = []
    limit = 1
    for i in range(n-1):
        p = random.uniform(0,limit)
        limit = limit - p
        Probs.append(p)
    Probs.append(limit)
    return Probs

def Probability_generator1(n):
    '''
    Generates list of Probabilites with n elements, by first generating n-1 probabilities.
    Then sorting them, add 0 and 1 to the list and we get list of Pobabilities that add up to 1 by taking difference 
    between elements of the previous list. Suggested by Dr. Banacki.
    '''
    
    Probs = []
    for i in range(n-1):
        R = random.uniform(0,1)
        if R not in [0,1]:
            Probs.append(R)
    Probs = sorted(Probs+[0,1])
    Probabilities = []
    for j in range(n):
        Probabilities.append(Probs[j+1]-Probs[j]) 
    return Probabilities

def Simplex_generator(n):
    P = Probability_generator1(n)
    simplex = np.identity(n)
    for i in range(n):
        simplex[i,i] = P[i]
    simplex = Qobj(simplex)
    return simplex

def Amplitude_Generator(Probs):
    for i in range(len(Probs)):
        Probs[i] = math.sqrt(Probs[i])
    return Probs

def Normalized_pure_state_generator(dim):
    simplex = Amplitude_Generator(Probability_generator1(dim))
    Q = [basis(dim, n) for n in range(dim)]
    NPS = []
    for i in range(dim):
        W = simplex[i]*Qobj(dimensions.collapse_dims_oper(tensor([Q[i],Q[i]])))
        NPS.append(W)
    NPS = sum(NPS)
    return(NPS)

def Arbitrary_Mixed_State(n):
    simplex = Simplex_generator(n)
    U = rand_unitary(n)
    AMS = U*simplex*U.dag()
    return AMS 

def Reshaping_Qobj(Q,subsys):
    K = len(subsys)
    N = np.prod(np.array(subsys))
    if N != Q.shape[0]:
        raise ValueError("Product of dimensions of subsystem and Total dimension of Qobj dont match")
    Q.dims = [subsys for i in range(len(subsys))]
    return Q
    
def Werner_Density_Matrix(dim,alpha):
    '''
    For an dim*dim system, Werner state is given as -
    l
    NPS = Normalized Pure State
    '''
    NPS = Normalized_pure_state_generator(dim)
    UV = tensor(rand_unitary(dim),rand_unitary(dim))
    print(UV)
    rho = alpha*UV*NPS*NPS.dag()*UV.dag() + (1-alpha)/(dim**2)*qeye(dim**2)
    return rho

print(Werner_Density_Matrix(3,0.5))
AMS = Arbitrary_Mixed_State(6)
AMS = Reshaping_Qobj(AMS,[2,3])
print(AMS)
partial_transpose(AMS, [False,True]).eigenenergies()

Quantum object: dims = [[3, 3], [3, 3]], shape = (9, 9), type = oper, isherm = False
Qobj data =
[[ 4.74641653e-01+0.59047602j  6.70447538e-02+0.04162546j
   2.09428605e-01-0.32071499j  3.55515314e-01-0.24585903j
   2.64410963e-02-0.03644413j -1.73910650e-01-0.13235024j
  -1.45166198e-01-0.08792894j -1.73034558e-02-0.00362486j
  -1.95100581e-02+0.08356275j]
 [ 7.26829134e-02+0.03073877j  7.87024371e-01+0.00728883j
  -1.84388946e-02+0.31759186j  2.04772875e-02-0.04009971j
   3.64662172e-02-0.44757594j  1.79976150e-01+0.02353468j
  -1.76560844e-02-0.00090103j -1.66235539e-01+0.05877652j
  -2.04559730e-02-0.06826946j]
 [ 2.86610110e-01-0.25411235j -1.47030665e-01+0.28211091j
   5.96170681e-01+0.35149714j -1.32839434e-01-0.17353727j
   1.54504371e-01+0.09525594j  2.24509070e-01-0.32483148j
  -4.08614473e-02+0.07545672j  9.33291999e-03-0.07065453j
  -1.52436028e-01-0.0283084j ]
 [-3.66104199e-01-0.22979451j -4.39165496e-02-0.00993192j
  -5.30687453e-02+0.21200285j  2.71770352e-01-0.55313639

TypeError: Incompatible Qobj shapes

In [427]:
# Helper function 1
'''
Helper functions - 
all the function that can be used across multiple modules 
and repeatedly often.
''' 

def generate_alpha(delta):
    alpha = random.uniform(0,2*cmath.pi*delta)
    return alpha

def generate_phi(r,s,delta):
    Xi = random.uniform(0,delta)
    phi = math.asin(Xi**(1/(2*r)))
    return phi

def generate_psi(r,s,delta):
    psi = random.uniform(0,2*cmath.pi*delta)
    return psi

def generate_chi(r,s,delta):
    chi = random.uniform(0,2*cmath.pi*delta)
    return chi

def para_i(r,s):
    return (s**2-2*r-1, s**2-2*r, (s+1)**2-2*(s+1)) 

In [428]:
# Helper functions 2

def Series_multiply(lst, order = 'normal'):
    Multiplied_list = []
    M = 1
    
    if order=='right':
        lst1 = lst.copy()
        lst1.reverse()
        
        for ls in lst1:
            if isinstance(ls,np.ndarray) and isinstance(M,np.ndarray):
                M = np.matmul(ls,M)
            else:
                M = ls*M
            Multiplied_list.append(M)

        Multiplied_list.reverse()
        
    elif order == 'left':
        for ls in lst:
            if isinstance(ls,np.ndarray) and isinstance(M,np.ndarray):
                M = np.matmul(M,ls)
            else:
                M = M*ls
            Multiplied_list.append(M)
            
    else:
        for ls in lst:
            if isinstance(ls,np.ndarray) and isinstance(M,np.ndarray):
                M = np.matmul(M,ls)
            else:
                M = M*ls
        return M
    
    return Multiplied_list

L = Series_multiply(lst,'left')

R = Series_multiply(lst,'right')

In [431]:
def tensor_product(matrices):
    if len(matrices) < 2:
        raise ValueError("At least two matrices are required.")

    return reduce(np.kron, matrices)

def tensor_product_left(tensors):
    if len(tensors) < 2:
        raise ValueError("At least two tensors are required.")

    result = [tensors[0]]
    for i in range(1, len(tensors)):
        result.append(np.kron(result[i-1], tensors[i]))

    return result

def tensor_product_right(tensors):
    if len(tensors) < 2:
        raise ValueError("At least two tensors are required.")

    result = [tensors[-1]]
    for i in range(len(tensors)-2, -1, -1):
        result.insert(0, np.kron(tensors[i], result[0]))

    return result

def multiply_right_matrices(matrices):
    if len(matrices) < 2:
        raise ValueError("At least two matrices are required.")

    result = [matrices[-1]]
    for i in range(len(matrices)-2, -1, -1):
        result.insert(0, np.matmul(matrices[i], result[0]))
        # Alternatively, you can use the '@' operator:
        # result.insert(0, matrices[i] @ result[0])

    return result

def Series_matrix_multiply(matrix_lst):
    return reduce(lambda a,b:np.matmul(a,b),matrix_lst)

In [432]:
# CUE parameter generator function -

def CUE_parameter_generator(size,delta):
    '''
    input: 
    size = N (size of one side of the random unitary matrix 
    being produced), depending on which, this function return a parameter 
    array of size. 
    
    delta = another user defined variable for determining random unitary
    
    return:
    Returns an array of size (n-1)^2 with the structure -
    [alpha,
    
    phi(1,2),
    psi(1,2),
    chi(1,2),
    
    phi(2,3),
    psi(2,3),
    phi(1,3),
    psi(1,3),
    chi(1,3),
    .
    .
    .
    phi(1,n),
    psi(1,n),
    chi(1,n),
    ]
    '''
    
    parameters = []
    parameters.append(('alpha',0,1,generate_alpha(delta)))
    for s in range(2,size+1):
        for r in range(s-1,0,-1):
            parameters.append(('phi',r,s,generate_phi(r,s,delta)))
            parameters.append(('psi',r,s,generate_psi(r,s,delta)))
            if r==1:
                parameters.append(('chi',r,s,generate_chi(r,s,delta)))
    return parameters

delta = 1
size = 3
parameters = np.array(CUE_parameter_generator(size,delta))
parameters

array([['alpha', '0', '1', '3.854789089248824'],
       ['phi', '1', '2', '0.4335060930382913'],
       ['psi', '1', '2', '3.614536685388278'],
       ['chi', '1', '2', '4.021195593879533'],
       ['phi', '2', '3', '1.2457247305439123'],
       ['psi', '2', '3', '4.929180996084024'],
       ['phi', '1', '3', '0.8071490327331502'],
       ['psi', '1', '3', '1.5836513569483326'],
       ['chi', '1', '3', '5.8359576685438315']], dtype='<U32')

In [433]:
def Elementary_Unitary_Matrices_Generator(size,r,s,delta,phi, psi, chi):
    # Creating the matrix
    E = np.eye(size, dtype=complex)
    r-=1 
    s-=1
    
    E[r,r] = cmath.cos(phi) * cmath.exp(complex(0,psi))  
    E[r,s] = cmath.sin(phi) * cmath.exp(complex(0,chi))
    E[s,r] = -cmath.sin(phi) * cmath.exp(complex(0,-chi))
    E[s,s] = cmath.cos(phi) * cmath.exp(complex(0,-psi))
    
    return E

n=3
a=1
b=2
delta = 1
phi = generate_phi(a,b,delta)
psi = generate_psi(a,b,delta)
chi = generate_chi(a,b,delta)
    
# Elementary_Unitary_Matrices_Generator(n,a,b,delta,phi,psi,chi)

In [720]:
# CUE_generator - 

# First step is to work with unitary matrices. 
# It will have a function to calculate random unitary matrix using Zyczkowski's method.


def CUE_generator(size, parameters, delta):
    '''
    input: 
    
    1) size = it is the dimensionality (of one side) of the aimed 
    random unitary matrix.
    
    2) parameters = This is a list of parameters used to parametrize the 
    unitary matrix, necessary to generate CUE(Circular Unitary Ensembles)
    deterministically. It is crucial to keep track of these parameters to
    eventually work with gradient descent.
    
    return:
    1) return a unitary matrix of given size, with given parameters and 
    delta.
    '''
    
    alpha = float(parameters[0][3])
    
    Left = []
    Elementary_matrices = [cmath.exp(complex(0,alpha))]
    right = []
    
    for s in range(2,size+1):
        for r in range(s-1,0,-1):
            indexes = para_i(r,s)
            phi = float(parameters[indexes[0]][3])
            psi = float(parameters[indexes[1]][3])
            if r==1:
                chi = float(parameters[indexes[2]][3])
            else:
                chi = 0
            E = Elementary_Unitary_Matrices_Generator(size,r,s,delta,phi,psi,chi)
            Elementary_matrices.append(E)
    
    
    Left_multiply = Series_multiply(Elementary_matrices,'left')
    Right_multiply = Series_multiply(Elementary_matrices,'right')
    U = Series_multiply(Elementary_matrices)
    
    if Qobj(U).check_isunitary():
        return (U,Elementary_matrices, Left_multiply, Right_multiply)
    else:
        raise Exception('U is not unitary.')
    
    
delta = 1
size = 3
parameters = CUE_parameter_generator(size, delta)

U = CUE_generator(size, parameters, delta)
U

(array([[-0.63307528-0.51744741j, -0.21071933+0.21654523j,
         -0.28699518-0.39724446j],
        [ 0.12958967+0.54693403j, -0.25195889+0.14786768j,
          0.0885694 -0.76868525j],
        [-0.06078725+0.10880467j, -0.35225163-0.83622735j,
         -0.40064199-0.02439374j]]),
 [(0.38820294150348494+0.9215739125040605j),
  array([[ 0.69028273+0.08172028j,  0.66597098-0.27076594j,
           0.        +0.j        ],
         [-0.66597098-0.27076594j,  0.69028273-0.08172028j,
           0.        +0.j        ],
         [ 0.        +0.j        ,  0.        +0.j        ,
           1.        +0.j        ]]),
  array([[ 1.        +0.j        ,  0.        +0.j        ,
           0.        +0.j        ],
         [ 0.        +0.j        ,  0.00596446+0.42024639j,
           0.90739043+0.j        ],
         [ 0.        +0.j        , -0.90739043-0.j        ,
           0.00596446-0.42024639j]]),
  array([[-0.86188721+0.41135474j,  0.        +0.j        ,
           0.23117499+0.1857305

In [435]:
# Calculating Y
def Y_Calculator(sep_state_set,U,rho):
    '''
    Inputs: 
    sep_state_set = list of seperable state vectors
    U_lst = the unitary to be applied for optimization
    rho = the state of the system being tested for entanglement, given as a density matrix
    
    Outputs:
    (Y,Y_list,left_Y, right_Y)
    where
    
    Y = The final calcuated value of Y, given value inputs.
    
    Y_list = The list of 'Y multiplier' 
    
    left_Y = The list of multiplied <Y(i,j)> from the left - 
    [(<Y(1,1)>), (<Y(1,1)><Y(1,2)>),....,(<Y(1,1)>...<Y(N,N)>)]
    
    right_Y = The list of multiplied <Y(i,j)> from the right -
    [(<Y(N,N)>), (<Y(N,N)><Y(N-1,N)>),....,(<Y(N,N)>...<Y(1,1)>)]
    
    '''
    U = Qobj(U)
    U_dag = U.dag()
    
    system_dim = len(sep_state_set)
    
    Y_list = []
    Yij_list = []
    for i in range(system_dim):
        for j in range(system_dim):
#             print(np.array(sep_state_set[i].dag().full()),np.array(U_dag.full()),np.array(rho.full()),np.array(U.full()),np.array(sep_state_set[j].full()))
            Y_multiplier = reduce(np.matmul,[np.array(sep_state_set[i].dag().full()),np.array(U_dag.full()),np.array(rho.full()),np.array(U.full()),np.array(sep_state_set[j].full())])
            Yij_list.append([Y_multiplier[0][0],i,j])
            Y_list.append(Y_multiplier[0][0])
    
    Y = reduce(lambda a,b:a*b, Y_list)
    left_Y = Series_multiply(Y_list,'left')
    right_Y = Series_multiply(Y_list,'right')
    
    return (Y,Yij_list,left_Y,right_Y)


#inputs:
system = [3,3]        
Sep_state_set = Seperable_state_set_generator(Mutually_Orthogonal_state_generator(system))

Hilbert_space_size = Series_multiply(system)
rho = rand_dm(Hilbert_space_size)

unitaries = []
Unitary_list = []
Total_U = []
for i in system:
    size = i
    delta = 1
    parameters = CUE_parameter_generator(size, delta)
    CUE = CUE_generator(size, parameters, delta)
    
    Unitary_list.append(CUE)
    Total_U.append(Qobj(CUE[0]))
U = np.array(tensor(Total_U))

Y_Calculator(Sep_state_set, U,rho)

[[Quantum object: dims = [[3], [1]], shape = (3, 1), type = ket
Qobj data =
[[1.]
 [0.]
 [0.]], Quantum object: dims = [[3], [1]], shape = (3, 1), type = ket
Qobj data =
[[0.]
 [1.]
 [0.]], Quantum object: dims = [[3], [1]], shape = (3, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [1.]]], [Quantum object: dims = [[3], [1]], shape = (3, 1), type = ket
Qobj data =
[[1.]
 [0.]
 [0.]], Quantum object: dims = [[3], [1]], shape = (3, 1), type = ket
Qobj data =
[[0.]
 [1.]
 [0.]], Quantum object: dims = [[3], [1]], shape = (3, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [1.]]]]


((4.11504414411815e-13+7.9433964907704e-29j),
 [[(0.13986290972879822-8.456776945386935e-18j), 0, 0],
  [(0.01578928332477348-0.027284575379724643j), 0, 1],
  [(-0.003359175833512609+0.011609359766680944j), 0, 2],
  [(0.01578928332477348+0.027284575379724636j), 1, 0],
  [(0.11447354015705236-6.938893903907228e-18j), 1, 1],
  [(0.026129411849391014-0.030806833737310375j), 1, 2],
  [(-0.003359175833512608-0.01160935976668095j), 2, 0],
  [(0.026129411849391+0.03080683373731037j), 2, 1],
  [(0.10851403310926229-2.168404344971009e-18j), 2, 2]],
 [(0.13986290972879822-8.456776945386935e-18j),
  (0.002208335108335212-0.0038161001033230183j),
  (3.688429307692815e-05+3.845630800340347e-05j),
  (-4.668874809191701e-07+1.613569817475793e-06j),
  (-5.344626279582562e-08+1.8471104929702283e-07j),
  (4.293843172734611e-09+6.472901212046673e-09j),
  (6.07224646860918e-11-7.159238349820452e-11j),
  (3.792176943579943e-12+8.077935669463161e-28j),
  (4.11504414411815e-13+7.9433964907704e-29j)],
 [(4.11

In [692]:
def calculate_dEdp(E,para,para_i,r,s):
    symbol = para[para_i][0]
    r = int(para[para_i][1])
    s = int(para[para_i][2])
    parameter_value = float(para[para_i][3])
    
    chi=0
    dEdp = np.eye(size, dtype=complex)
    r-=1 
    s-=1
        
    if symbol == 'phi':
        phi = parameter_value
        psi = float(para[para_i+1][3])
        if r==1:
            chi = float(para[para_i+2][3])

        dEdp[r,r] = -cmath.sin(phi) * cmath.exp(complex(0,psi))  
        dEdp[r,s] = cmath.cos(phi) * cmath.exp(complex(0,chi))
        dEdp[s,r] = -cmath.cos(phi) * cmath.exp(complex(0,-chi))
        dEdp[s,s] = -cmath.sin(phi) * cmath.exp(complex(0,-psi))
        
    elif symbol == 'psi':
        phi = float(para[para_i-1][3])
        psi = parameter_value
        if r==1:
            chi = float(para[para_i+1][3])
    
        dEdp[r,r] = complex(0,1) * cmath.cos(phi) * cmath.exp(complex(0,psi))  
        dEdp[r,s] = cmath.sin(phi) * cmath.exp(complex(0,chi))
        dEdp[s,r] = -cmath.sin(phi) * cmath.exp(complex(0,-chi))
        dEdp[s,s] = cmath.cos(phi) * cmath.exp(complex(0,-psi))
    
    elif symbol == 'chi':
        phi = float(para[para_i-2][3])
        psi = float(para[para_i-1][3])
        if r==1:
            chi = parameter_value
    
        dEdp[r,r] = cmath.cos(phi) * cmath.exp(complex(0,psi))  
        dEdp[r,s] = cmath.sin(phi) * cmath.exp(complex(0,chi))
        dEdp[s,r] = -cmath.sin(phi) * cmath.exp(complex(0,-chi))
        dEdp[s,s] = cmath.cos(phi) * cmath.exp(complex(0,-psi))
        
    return dEdp

In [726]:
def calculate_du_system_dp(u_system, para,r,s,para_i,sys_i):
    symbol = para[para_i][0]
    r = float(para[para_i][1])
    s = float(para[para_i][2])
    parameter_value = float(para[para_i][3])
    
    elementary_index = int((s-1)*(s)/2 - (r-1) - 1)
    E = u_system[1][elementary_index]
    
    if symbol=='alpha':
        return complex(0,1)*parameter_value*u_system[0]
    
    dEdp = calculate_dEdp(E,para,para_i,r,s)
    
    if elementary_index == 0:
        du_system_dp = Series_multiply([dEdp,u_system[3][sys_i+1]])
    elif elementary_index == len(u_system[1])-1:
        du_system_dp = Series_multiply([u_system[2][elementary_index-1],dEdp])
    else:
        du_system_dp = Series_multiply([u_system[2][sys_i-1],dEdp,u_system[3][sys_i+1]])
        
    return du_system_dp

In [727]:
def calculate_dUdp(U,para,r,s,sys_i,para_i,Total_U,right_tensor_product,left_tensor_product):
    u_system = U[sys_i]
    du_system_dp = calculate_du_system_dp(u_system, para,r,s,para_i,sys_i)
    
    if sys_i == 0:
        dUdp = tensor_product([du_system_dp,right_tensor_product[sys_i+1]])
    elif sys_i == len(U)-1:
        dUdp = tensor_product([left_tensor_product[sys_i-1],du_system_dp])
    else:
        dUdp = tensor_product([left_tensor_product[sys_i-1],du_system_dp,right_tensor_product[sys_i+1]])
    
    return dUdp

In [803]:
def calculate_dYij_dp(sep_state_set,rho,U,para,r,s,sys_i,para_i,xi_a,xi_b,Total_U,right_tensor_product,left_tensor_product):
    Total_U_dag = np.array(Total_U).transpose().conjugate()
    xi_a_dag = np.array(xi_a).transpose().conjugate()
      
    dUdp = calculate_dUdp(U,para,r,s,sys_i,para_i,Total_U,right_tensor_product,left_tensor_product)
    
    b = (-Series_matrix_multiply([dUdp,Total_U_dag,np.array(rho),Total_U])+np.matmul(np.array(rho),dUdp))
    a = [Total_U_dag,b]
    Ut = Series_matrix_multiply(a)
    
    dYij_dp = Series_matrix_multiply([xi_a_dag,Ut,np.array(xi_b)])
    return dYij_dp[0][0]

In [842]:
def calculate_dY_dp(sep_state_set,rho,U,para,r,s,Y,sys_i,para_i,Total_U,right_tensor_product,left_tensor_product):
    
    partial_diff_sum = 0
    for y in Y[1]:
        xi_a = sep_state_set[y[1]]
        xi_b = sep_state_set[y[2]]
        dYij_dp = calculate_dYij_dp(sep_state_set,rho,U,para,r,s,sys_i,para_i,xi_a,xi_b,Total_U,right_tensor_product,left_tensor_product)
        partial_diff_sum+=((dYij_dp)/(y[0]))
    
    # N calculator
    N = (len(sep_state_set))**2
    
    #Y 
    Y = Y[0]
    print(Y)
    
    dYdp = partial_diff_sum*(Y**(1/N))/N
    return dYdp

In [843]:
def Gradient(sep_state_set,rho,U,parameters):
    '''
    U is a list of local unitaries = [U_1,U_2,....,U_k]
    '''
    Local_unitary_list = [U[n][0] for n in range(len(U))]
    
    Total_U = tensor_product(Local_unitary_list)
    right_tensor_product = tensor_product_right(Local_unitary_list)
    left_tensor_product = tensor_product_left(Local_unitary_list)
    
    Y = Y_Calculator(Sep_state_set, Total_U, rho)
    
    gradient = []
    for sys_i in range(len(parameters)):
        gradient.append([])
        for para_i in range(len(parameters[sys_i])):
            r = parameters[sys_i][para_i][1]
            s = parameters[sys_i][para_i][2]
            para = parameters[sys_i]
            dYdp = calculate_dY_dp(sep_state_set,rho,U,para,r,s,Y,sys_i,para_i,Total_U,right_tensor_product,left_tensor_product)
            gradient[sys_i].append(dYdp)
            
    return gradient

#inputs:
system = [3,3]        
sep_state_set = Seperable_state_set_generator(Mutually_Orthogonal_state_generator(system))

Hilbert_space_size = Series_multiply(system)
rho = rand_dm(Hilbert_space_size)

unitaries = []
delta = 1
parameters = []
for n in system:
    size = n
    
    p = CUE_parameter_generator(size, delta)
    parameters.append(p)
    
    uni = CUE_generator(size, p, delta)
    unitaries.append(uni)
    
pprint(parameters) 
Gradient(sep_state_set,rho,unitaries,parameters)

[[('alpha', 0, 1, 5.256791260720622),
  ('phi', 1, 2, 0.24344463786413803),
  ('psi', 1, 2, 4.540412101622886),
  ('chi', 1, 2, 3.1251426351878666),
  ('phi', 2, 3, 0.721880150546739),
  ('psi', 2, 3, 2.522474218184489),
  ('phi', 1, 3, 0.527092507912404),
  ('psi', 1, 3, 0.9837340993760962),
  ('chi', 1, 3, 0.7034314709496075)],
 [('alpha', 0, 1, 0.19773240792584032),
  ('phi', 1, 2, 0.44622299820993216),
  ('psi', 1, 2, 4.725348694776005),
  ('chi', 1, 2, 1.1373078784586692),
  ('phi', 2, 3, 0.814939147693479),
  ('psi', 2, 3, 4.590237222038209),
  ('phi', 1, 3, 1.1041157615010893),
  ('psi', 1, 3, 2.571487291217492),
  ('chi', 1, 3, 0.8566144626808848)]]
(3.378735605181368e-14-6.830152942757927e-30j)
(3.378735605181368e-14-6.830152942757927e-30j)
(3.378735605181368e-14-6.830152942757927e-30j)
(3.378735605181368e-14-6.830152942757927e-30j)
(3.378735605181368e-14-6.830152942757927e-30j)
(3.378735605181368e-14-6.830152942757927e-30j)
(3.378735605181368e-14-6.830152942757927e-30j)
(3.37

[[(1.2341531520484146e-17+2.969107327255196e-17j),
  (-0.01748533454184003+0.014741946865533054j),
  (-0.00877740716904263+0.02572422567174194j),
  (-0.016887178450771825+0.025163606857730534j),
  (-0.0017919290028841858-0.005417718113839454j),
  (0.0018971874399961767+0.011012496966898967j),
  (0.00468405642193016+0.022079914297179263j),
  (-0.015495753084941279+0.008704650489686206j),
  (-0.018887739143174643+0.004339277377426976j)],
 [(7.189902480251699e-19-7.229498928826111e-19j),
  (0.02339848958691875-0.016229775031465835j),
  (0.014050958772718686-0.002110061976745291j),
  (0.012072139424395334-0.0034357194643865408j),
  (-0.0017592137192216067+0.010595821485972826j),
  (0.017469527763222272+0.00429337925766392j),
  (0.010065284703277723-0.013350978894943875j),
  (-0.004365421911565617-0.008238775433789583j),
  (-0.0046071604261199986-0.007075668834142589j)]]

In [838]:
def adjusted_parameters(parameters, gradient, stepsize):
    new_parameters = []
    
    for system in range(len(gradient)):
        for parameter in range(len())
    
    

In [839]:
def Gradient_Descent(system,rho):
    sep_state_set = Seperable_state_set_generator(Mutually_Orthogonal_state_generator(system))
    
    unitaries = []
    delta = 1
    parameters = []
    for n in system:
        size = n
    
        p = CUE_parameter_generator(size, delta)
        parameters.append(p)
    
        uni = CUE_generator(size, p, delta)
        unitaries.append(uni)
    
    gradient = Gradient(sep_state_set,rho,unitaries,parameters)    
    pprint(parameters)
    pprint(gradient)
    
    stepsize = 0.1
    
    new_parameters = adjusted_parameters(parameters, gradient, stepsize)
    
    return gradient

In [840]:
#inputs:
system = [3,3]        

Hilbert_space_size = Series_multiply(system)
rho = rand_dm(Hilbert_space_size)

Gradient_Descent(system,rho)

[[('alpha', 0, 1, 1.9265599003595386),
  ('phi', 1, 2, 0.7016699749409243),
  ('psi', 1, 2, 3.030452103115612),
  ('chi', 1, 2, 4.685212186949861),
  ('phi', 2, 3, 1.138036342334245),
  ('psi', 2, 3, 2.550830254757267),
  ('phi', 1, 3, 1.3146211486162565),
  ('psi', 1, 3, 5.36274161314628),
  ('chi', 1, 3, 3.3724039183635575)],
 [('alpha', 0, 1, 2.660652782160492),
  ('phi', 1, 2, 1.0364156869022596),
  ('psi', 1, 2, 0.24633766882956562),
  ('chi', 1, 2, 2.1353340910171665),
  ('phi', 2, 3, 0.8887176501111057),
  ('psi', 2, 3, 6.102991181814834),
  ('phi', 1, 3, 1.292837969244448),
  ('psi', 1, 3, 4.162245686893497),
  ('chi', 1, 3, 1.8432759978280007)]]
[[1.3148182765626012e-17,
  0.015167401678298,
  0.01966849552393541,
  0.011511553294444046,
  0.02129999418127391,
  0.010075493358408164,
  0.014839594715727943,
  0.0016043391059761144,
  0.0037519246446362396],
 [1.943827218937582e-17,
  0.02605690950630648,
  0.032798693636673315,
  0.016311493414799313,
  0.04018508796602308,
  

[[1.3148182765626012e-17,
  0.015167401678298,
  0.01966849552393541,
  0.011511553294444046,
  0.02129999418127391,
  0.010075493358408164,
  0.014839594715727943,
  0.0016043391059761144,
  0.0037519246446362396],
 [1.943827218937582e-17,
  0.02605690950630648,
  0.032798693636673315,
  0.016311493414799313,
  0.04018508796602308,
  0.012724169870990574,
  0.043305959128390574,
  0.017557551038800336,
  0.01626567664770857]]